# Part I. ETL Pipeline for Pre-Processing the Files

The following code pre-processes the csv files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now we will work with the csv file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## The Apache Cassandra code is written below

#### Creating a Cluster

In [5]:
# Makes a connection to a Cassandra instance on the local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1']) 

# To establish connection and begin executing queries, a session is needed
session = cluster.connect()

#### Create Keyspace
A keyspace in Apache Cassandra is basically a collection of tables. It is a data container in Cassandra, similar to a database in relational database management system. 

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify_events 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify_events')
except Exception as e:
    print(e)

### Now we need to create tables to run the given 3 queries. With Apache Cassandra it is important to model the database tables on the queries one wants to run.

## Below queries are created to answer the following questions

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
- Here the query would look like this: 'SELECT artist, song, length FROM song_details_by_session WHERE sessionId = 338 AND itemInSession = 4'
- Since we use sessionId and itemInSession in the WHEN clause we should use these as primary key - therefore as a composite primary key consisting of the two columns


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
- Here the query would look like this: 'SELECT artist, song, firstName, lastName FROM song_user_info_by_userid_sessionid WHERE userId = 10 AND sessionId = 182'
- As we use userId and sessionId in the WHERE clause we want those two to be the composite partition key in the primary key
- Furthermore we want to sort by itemInSession, therefore we want this column to be our clustering column

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
- Here the query would look like this: "SELECT firstName, lastName FROM user_name_by_song WHERE song = 'All Hands Against His Own'"
- As we have song in our WHERE clause we want it to be our primary key
- But since song is not usable as a unique key - we need another column to make the key unique
- userId is chosen as the clustering column to provide a unique key

#### Modelling the query for the first question:

In [8]:
query_1 = "CREATE TABLE IF NOT EXISTS song_details_by_session "
query_1 = query_1 + "(sessionId INT, itemInSession INT, artist VARCHAR, song VARCHAR, length FLOAT, PRIMARY KEY (sessionId, itemInSession))"

try:
    session.execute(query_1)
except Exception as e:
    print(e)

#### Inserting records from the csv file:

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_details_by_session (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## For e.g., to INSERT artist_name and user first_name the code would be `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Perform the SELECT from question 1 to verify that the data have been inserted into each table:

In [10]:
query = 'SELECT artist, song, length FROM song_details_by_session WHERE sessionId = 338 AND itemInSession = 4'

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### Therefore the answer to the first question is: Faithless Music Matters (Mark Knight Dub) 495.30731201171875

### The steps will be repeated for question 2 and 3 below.

In [11]:
query_2 = "CREATE TABLE IF NOT EXISTS song_user_info_by_userid_sessionid "
query_2 = query_2 + "(userId INT, sessionId INT, itemInSession INT, artist VARCHAR, song VARCHAR, firstName VARCHAR, lastName VARCHAR, PRIMARY KEY ((userId, sessionId), itemInSession))"

try:
    session.execute(query_2)
except Exception as e:
    print(e)       

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_user_info_by_userid_sessionid (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
query = 'SELECT artist, song, firstName, lastName FROM song_user_info_by_userid_sessionid WHERE userId = 10 AND sessionId = 182'

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Therefore the answer for question 2 is to be found above as the query result

In [14]:
query_3 = "CREATE TABLE IF NOT EXISTS user_name_by_song "
query_3 = query_3 + "(song VARCHAR, userId INT, firstName VARCHAR, lastName VARCHAR, PRIMARY KEY (song, userId))"

try:
    session.execute(query_3)
except Exception as e:
    print(e)

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_name_by_song (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [16]:
query = "SELECT firstName, lastName FROM user_name_by_song WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


#### The above query-outcome shows the answer to question 3

### Now the tables are being dropped and the session closed afterwards

In [17]:
drop_t1 = "DROP TABLE song_details_by_session"
try:
    rows = session.execute(drop_t1)
except Exception as e:
    print(e)
    
drop_t2 = "DROP TABLE song_user_info_by_userid_sessionid"
try:
    rows = session.execute(drop_t2)
except Exception as e:
    print(e)

drop_t3 = "DROP TABLE user_name_by_song"
try:
    rows = session.execute(drop_t3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()